In [18]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pmdarima.arima import auto_arima

# Load the data into a DataFrame
# Replace 'df' with your actual DataFrame
df = pd.read_csv('./training_datasets/Bitcoin [BTC]_15mprice_history.csv')

# Convert the 'timestamp' column to datetime
df['timestamp'] = pd.to_datetime(df['DateTime'])
df = df.drop(['DateTime','Closing Price', 'High', 'Low'], axis=1)
df = df.set_index('timestamp') # .resample('15T').mean()
df.rename(columns={'Open':'price'}, inplace=True)
df.head()

,price
timestamp,
2023-05-24 15:00:00,26254.93
2023-05-24 15:15:00,26150.01
2023-05-24 15:30:00,26322.22
2023-05-24 15:45:00,26298.77
2023-05-24 16:00:00,26314.69


In [28]:
FORECAST_PERIOD = 5
FREQ = '15T' # '1H'

df1 = df[-FORECAST_PERIOD:].copy()
df = df[:-FORECAST_PERIOD]

# Fit the ARIMA model using pmdarima's auto_arima
model = auto_arima(df['price'],
                   d=None, 
                   method='lbfgs', 
                   seasonal=False,
                   trace=True,
                   stationary=False,
                   information_criterion='aic',
                   trend=None)

# Forecast the next FORECAST_PERIOD steps (4x15 minutes)
forecast, conf_int = model.predict(n_periods=FORECAST_PERIOD, return_conf_int=True)

# Create a datetime index for the forecasted periods
forecast_index = pd.date_range(start=df.index[-1], periods=FORECAST_PERIOD, freq='15T')

# Create a trace for the actual data
trace_actual = go.Scatter(
    x=df.index,
    y=df['price'],
    name='Actual'
)

# Create a trace for the forecasted data
trace_forecast = go.Scatter(
    x=forecast_index,
    y=forecast,
    name='Forecast'
)

# Create a trace for the upper and lower confidence intervals
trace_upper = go.Scatter(
    x=forecast_index,
    y=conf_int[:, 1],
    fill=None,
    mode='lines',
    line=dict(color='rgba(68, 68, 68, 0.3)'),
    name='Upper CI'
)

trace_lower = go.Scatter(
    x=forecast_index,
    y=conf_int[:, 0],
    fill='tonexty',
    mode='lines',
    line=dict(color='rgba(68, 68, 68, 0.3)'),
    name='Lower CI'
)

# Create the plotly figure
fig = go.Figure(data=[trace_actual, trace_forecast, trace_upper, trace_lower])


# Set the plotly layout
fig.update_layout(
    title='Bitcoin Price Forecast',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    showlegend=True
)

# Actual Forecast
trace_actual_prices = go.Scatter(
    x=df1.index,
    y=df1['price'],
    name='Actual Forecast'
)
fig.add_trace(trace_actual_prices)


# Display the plot
fig.show()

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=9684.737, Time=1.12 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=9691.350, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=9691.525, Time=0.12 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=9691.352, Time=0.20 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=9689.820, Time=0.05 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=9693.918, Time=0.43 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=9693.604, Time=0.61 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=9686.296, Time=1.86 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=9691.316, Time=0.90 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=9692.795, Time=0.32 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=9694.812, Time=0.73 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=9694.662, Time=1.05 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=9688.532, Time=1.49 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=9683.341, Time=0.66 sec
 ARIMA(1,1,2)(0,0,0)[0]          